In [1]:
## Callin Switzer
## Capture Images from pt grey camera

In [2]:
# setup arduino
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2

import datetime

import PyCapture2 as fc2
import sys

import time

import os
import peakutils 
import msvcrt
import winsound
import shutil
import pandas as pd
import re

import skimage.io as io

In [3]:
print("last update: "  +  str(datetime.datetime.now()))

last update: 2018-01-29 14:10:35.088027


In [4]:
# capture image with point grey camera
bus = fc2.BusManager()
numCams = bus.getNumOfCameras()
print("Number of cameras detected: ", numCams)
if not numCams:
    print("Insufficient number of cameras. Exiting...")
    exit()

Number of cameras detected:  2


In [5]:
def enableEmbeddedTimeStamp(cam, enableTimeStamp):
    embeddedInfo = cam.getEmbeddedImageInfo()
    if embeddedInfo.available.timestamp:
        cam.setEmbeddedImageInfo(timestamp = enableTimeStamp)
        if(enableTimeStamp):
            print("\nTimeStamp is enabled.\n")
        else:
            print("\nTimeStamp is disabled.\n")

In [6]:
# open qtconsole if it's not already running
try:
    console_is_running
except NameError:
    console_is_running = True
    %qtconsole

In [7]:
def printCameraInfo(cam):
    camInfo = cam.getCameraInfo()
    print("\n*** CAMERA INFORMATION ***\n")
    print("Serial number - ", camInfo.serialNumber)
    print("Camera model - ", camInfo.modelName)
    print("Camera vendor - ", camInfo.vendorName)
    print("Sensor - ", camInfo.sensorInfo)
    print("Resolution - ", camInfo.sensorResolution)
    print("Firmware version - ", camInfo.firmwareVersion)
    print("Firmware build time - ", camInfo.firmwareBuildTime)
    fRateProp = cam.getProperty(fc2.PROPERTY_TYPE.FRAME_RATE)
    print("FrameRate - ", fRateProp.absValue)
    print()

In [8]:
c = fc2.Camera()
c.connect(bus.getCameraFromIndex(0))
printCameraInfo(c)

d = fc2.Camera()
d.connect(bus.getCameraFromIndex(1))
printCameraInfo(d)


*** CAMERA INFORMATION ***

Serial number -  16060682
Camera model -  b'Chameleon3 CM3-U3-13Y3C'
Camera vendor -  b'Point Grey Research'
Sensor -  b'OnSemi PYTHON1300 (1/2" Color CMOS)'
Resolution -  b'1280x1024'
Firmware version -  b'1.9.3.0'
Firmware build time -  b'Mon Jul 27 20:45:52 2015'
FrameRate -  92.39653778076172


*** CAMERA INFORMATION ***

Serial number -  16061221
Camera model -  b'Chameleon3 CM3-U3-13Y3C'
Camera vendor -  b'Point Grey Research'
Sensor -  b'OnSemi PYTHON1300 (1/2" Color CMOS)'
Resolution -  b'1280x1024'
Firmware version -  b'1.9.3.0'
Firmware build time -  b'Mon Jul 27 20:45:52 2015'
FrameRate -  92.39653778076172



In [9]:
def img2array(image):
    return(np.array(image.getData(), dtype="uint8").reshape( (image.getRows(), image.getCols()) ))

In [10]:
# start recording video
def saveAviHelper2(cam, cam2, fileFormat, fileName, fileName2, frameRate, maxImgs = 500):
    
    numImages = 0

    avi = fc2.AVIRecorder()
    avi2 = fc2.AVIRecorder()

    for i in range(maxImgs):
        
        try:
            image = cam.retrieveBuffer()
            image2 = cam2.retrieveBuffer()
        except fc2.Fc2error as fc2Err:
            print("Error retrieving buffer : ", fc2Err)
            continue

        print("Grabbed image {}".format(i))

        if (i == 0):
            if fileFormat == "AVI":
                avi.AVIOpen(fileName, frameRate)
                avi2.AVIOpen(fileName2, frameRate)
#             elif fileFormat == "MJPG":
#                 avi.MJPGOpen(fileName, frameRate, 75)
#             elif fileFormat == "H264":
#                 avi.H264Open(fileName, frameRate, image.getCols(), image.getRows(), 1000000)
            else:
                print("Specified format is not available.")
                return
            
            # show still image
            img = np.concatenate((img2array(image), img2array(image2)), axis = 1)

            # Display the resulting frame
            cv2.imshow('image', img)

        # break when "q" is pressed on keyboard
        if cv2.waitKey(1) & 0xFF == ord('q'):
            for jj in range(10):
                cv2.destroyAllWindows()
            break

        # refref add image timestamp

        avi.append(image)
        avi2.append(image2)
        numImages += 1
        print("Appended image {}...".format(i))

    # close windows if loop ends
    for jj in range(10):
        cv2.destroyAllWindows()
        
    print("Appended {} images to {} file: {}...".format(numImages, fileFormat, fileName))
    avi.close()
    avi2.close()

In [11]:
c = fc2.Camera()
c.connect(bus.getCameraFromIndex(0))
enableEmbeddedTimeStamp(c, True)
c.startCapture()


d = fc2.Camera()
d.connect(bus.getCameraFromIndex(1))
enableEmbeddedTimeStamp(c, True)
d.startCapture()



TimeStamp is enabled.


TimeStamp is enabled.



In [12]:
# display images via live preview

def livePreview2():
    cv2.namedWindow('image',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('image', 1000,800)
    while(True):
        stt = time.time()
        # Capture frame-by-frame
        image = c.retrieveBuffer()
        image2 = d.retrieveBuffer()
        img = np.concatenate((img2array(image), img2array(image2)), axis = 1)

        # Our operations on the frame come here
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Display the resulting frame
        cv2.imshow('image', img)
        
        print(time.time() - stt)

        # break when "q" is pressed on keyboard
        if cv2.waitKey(1) & 0xFF == ord('q'):
            for jj in range(10):
                cv2.destroyAllWindows()
            break



In [20]:
livePreview2()

0.3360557556152344
0.291074275970459
0.2949066162109375
0.28728580474853516
0.29521918296813965
0.2834787368774414
0.2830348014831543
0.2699460983276367
0.30141282081604004
0.29102349281311035
0.28218793869018555
0.28177928924560547
0.2825627326965332
0.2857358455657959
0.28440403938293457
0.28283071517944336
0.2817094326019287
0.28400611877441406
0.2890660762786865
0.28887319564819336
0.2801401615142822
0.28720688819885254
0.2834005355834961
0.28602075576782227
0.2981553077697754
0.28829431533813477
0.28029704093933105
0.28823161125183105
0.2836029529571533
0.28745603561401367
0.2879042625427246
0.2841808795928955
0.2862682342529297
0.2855405807495117
0.2839221954345703
0.2864990234375
0.28854870796203613
0.2859220504760742
0.28170156478881836
0.28102612495422363
0.2841055393218994
0.28187036514282227
0.28290486335754395
0.28230714797973633
0.28365111351013184
0.28482675552368164
0.2817971706390381
0.2827765941619873
0.28061819076538086
0.28578734397888184
0.28581929206848145
0.284433

In [23]:
directory = os.path.join("C:\\Users\\Combes4\Desktop\\temp3")

movieID = str(datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S_%f")[:-3])
fileName = os.path.join(directory,   movieID + "_cam1" +".avi")
fileName2 = os.path.join(directory,  movieID + "_cam2" +".avi")
saveAviHelper2(c,d, "AVI", fileName.encode("utf-8"), fileName2.encode("utf-8"), 10, maxImgs = 10000)

Grabbed image 0
Appended image 0...
Grabbed image 1
Appended image 1...
Grabbed image 2
Appended image 2...
Grabbed image 3
Appended image 3...
Grabbed image 4
Appended image 4...
Grabbed image 5
Appended image 5...
Grabbed image 6
Appended image 6...
Grabbed image 7
Appended image 7...
Grabbed image 8
Appended image 8...
Grabbed image 9
Appended image 9...
Grabbed image 10
Appended image 10...
Grabbed image 11
Appended image 11...
Grabbed image 12
Appended image 12...
Grabbed image 13
Appended image 13...
Grabbed image 14
Appended image 14...
Grabbed image 15
Appended image 15...
Grabbed image 16
Appended image 16...
Grabbed image 17
Appended image 17...
Grabbed image 18
Appended image 18...
Grabbed image 19
Appended image 19...
Grabbed image 20
Appended image 20...
Grabbed image 21
Appended image 21...
Grabbed image 22
Appended image 22...
Grabbed image 23
Appended image 23...
Grabbed image 24
Appended image 24...
Grabbed image 25
Appended image 25...
Grabbed image 26
Appended image 

In [ ]:
# When everything done, release the capture
c.stopCapture()
c.disconnect()

d.stopCapture()
d.disconnect()
cv2.destroyAllWindows()

In [ ]:
os.getcwd()

In [ ]:
fileName

In [16]:
# combine videos to see if they're both the same

vidPath =  re.sub(".avi", "-0000.avi", fileName)
vidPath2 = re.sub(".avi", "-0000.avi", fileName2)

cap = cv2.VideoCapture(vidPath)
cap2 = cv2.VideoCapture(vidPath2)

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

print( length, width, height, fps)

cap.release()
cap2.release()

3838 1280 1024 10.0


In [17]:
print(vidPath,"\n", vidPath2)

C:\Users\Combes4\Desktop\temp3\2018_01_29__14_38_49_011_cam1-0000.avi 
 C:\Users\Combes4\Desktop\temp3\2018_01_29__14_38_49_011_cam2-0000.avi


In [18]:
from subprocess import call
os.chdir(directory)
os.getcwd()

outputMovie = output = re.sub("cam2-0000.avi", "combined.mp4", vidPath2)

In [19]:

# combine vids into one

call(["ffmpeg", "-i",
      vidPath, 
      "-i",
      vidPath2, 
      "-filter_complex",
      '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]', 

  "-map", "[vid]", 
  "-c:v", "libx264", 
  outputMovie]) 

0

In [ ]:
vid2.split(" ")[1].split(".avi")[0] + ".mp4"

In [ ]:
import os
from subprocess import call


vid1 = "2018-01-26 14_02_48.315498_cam1-0000.avi"
vid2 = "2018-01-26 14_02_48.315498_cam2-0000.avi"
output = re.sub("cam2-0000.avi", "combined.mp4", vid2)
output

In [ ]:
vid1 = "SaveImageToAviEx12018-01-26 13_43_30.916822-0000.avi"
vid2 = "SaveImageToAviEx22018-01-26 13_43_30.916822-0000.avi"
output = re.sub("-0000.avi", "combined.mp4", vid2)
output

In [ ]:
vid1 = "SaveImageToAviEx12018-01-26 13_39_47.374097-0000.avi"
vid2 = "SaveImageToAviEx22018-01-26 13_39_47.374097-0000.avi"
output = re.sub("-0000.avi", "combined.mp4", vid2)
output

In [ ]:
call(["ffmpeg", "-i",
      vid1, 
      "-i",
      vid2, 
      "-filter_complex",
      '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]', 

  "-map", "[vid]", 
  "-c:v", "libx264", 
#      "-r", "10",
  output]) 

In [ ]:
import os
from subprocess import call

directory = os.path.join("C:\\Users\\Combes4\Desktop\\twoCamSave2")
os.chdir(directory)
os.chdir("C:\\Users\\Combes4\\Desktop\\twoCamSave2")

vid1 = "SaveImageToAviEx12018-01-24 14_38_28.512556-0000.avi"
vid2 = "SaveImageToAviEx22018-01-24 14_38_28.512556-0000.avi"
output = vid2.split(" ")[1].split(".avi")[0] + ".mp4"

call(["ffmpeg", "-i",
      vid1, 
      "-i",
      vid2, 
      "-filter_complex",
      '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]', 

  "-map", "[vid]", 
  "-c:v", "libx264", 
#      "-r", "10",
  output]) 

In [ ]:
import os
from subprocess import call

directory = os.path.join("C:\\Users\\Combes4\Desktop\\twoCamSave2")
os.chdir(directory)
os.chdir("C:\\Users\\Combes4\\Desktop\\twoCamSave2")

vid1 = "SaveImageToAviEx12018-01-24 14_40_20.342783-0000.avi"
vid2 = "SaveImageToAviEx22018-01-24 14_40_20.342783-0000.avi"
output = vid2.split(" ")[1].split(".avi")[0] + ".mp4"

call(["ffmpeg", "-i",
      vid1, 
      "-i",
      vid2, 
      "-filter_complex",
      '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]', 

  "-map", "[vid]", 
  "-c:v", "libx264", 
#      "-r", "10",
  output]) 

In [ ]:
import os
from subprocess import call

directory = os.path.join("C:\\Users\\Combes4\Desktop\\twoCamSave2")
os.chdir(directory)
os.chdir("C:\\Users\\Combes4\\Desktop\\twoCamSave2")

vid1 = "SaveImageToAviEx12018-01-24 14_41_16.748791-0000.avi"
vid2 = "SaveImageToAviEx22018-01-24 14_41_16.748791-0000.avi"
output = vid2.split(" ")[1].split(".avi")[0] + ".mp4"

call(["ffmpeg", "-i",
      vid1, 
      "-i",
      vid2, 
      "-filter_complex",
      '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]', 

  "-map", "[vid]", 
  "-c:v", "libx264", 
#      "-r", "10",
  output]) 

In [ ]:
## This doesn't work asynchronously -- may have to use multiprocessing

import os
from subprocess import call

call(["python", "006_twoCameraCapture.py"])

In [ ]:
print("hello")

In [ ]:
import subprocess

In [ ]:
import os
print(os.popen("echo Hello, World!").read())

In [ ]:
import asyncio

In [ ]:
import asyncio

def hello_world(loop):
    for ii in range(10000):
        print('Hello World')
    loop.stop()
    
loop = asyncio.new_event_loop()    
asyncio.set_event_loop(asyncio.new_event_loop())

# Schedule a call to hello_world()
loop.call_soon(hello_world, loop)


loop.run_until_complete(loop.shutdown_asyncgens())
loop.close()

In [ ]:
loop.close()

In [ ]:
from multiprocessing import Process

import time


len_x = 10000000
x = [10]*len_x
x[:15]

def squareList(nums):
    for i in nums:
        squared(i)

num_pros = 4

pro_list = []
x3 = [10]*(len_x//num_pros)
for p in range(num_pros):
    p = Process(target=squareList, args=(x3,))
    pro_list.append(p)

start = time.time()
for p in pro_list:
    p.start()

for p in pro_list:
    p.join()

squareprocesstime = time.time() - start
print("Squaring 10 million numbers took {} seconds with 4 processes.".format(squareprocesstime))

# combine vids with ffmpeg
ffmpeg \
  -i input1.mp4 \
  -i input2.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map [vid] \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  output.mp4

In [ ]:

ctr = 0
photoDir = os.path.join("C:\\Users\\Combes4\Desktop\\twoCamSave2\\", movieID)

# make directory, if it doesn't exist
if not os.path.exists(photoDir):
    os.makedirs(photoDir)

    
    #refref" speed this up with opencv
for ff in range(length):
    ret, frame = cap.read()
    ret2, frame2 = cap2.read()
    im2 = np.concatenate((frame, frame2), axis = 1)
    io.imsave(os.path.join(photoDir, str(ctr).zfill(4) + '.png'), im2)
    print(ff)
    ctr += 1

In [ ]:
# save images as video
from subprocess import call
os.chdir(photoDir)

call(["ffmpeg", "-start_number", "0", "-i", "%04d.png", "-r", "10",  "-c:v", "h264", "-pix_fmt", "yuv420p", "-y", "combinedVid.mp4"]) 